# Прогнозирование: количественные и качественные методы

Добро пожаловать! В этом ноутбуке:
1. **Изучим**, зачем нужны прогнозы и **какие** бывают методы (количественные и качественные).
2. **Научимся** применять на Python:
   - скользящее среднее (MA);
   - экспоненциальное сглаживание (SES, метод Хольта);
   - трендовые регрессии (линейная);
3. **Поговорим** о качественных методах (экспертные оценки, сценарии, SWOT) — зачем они нужны.
4. **Решим расширенные задачи** (по три задания для каждого уровня: простой, стандартный, продвинутый), а потом увидим **решения**.

**Совет:** после чтения теории или запуска кода задавайте себе вопросы: *«А что, если я изменю параметр \(\alpha\)? Как метод реагирует?»*, *«Почему скользящее среднее отстаёт при тренде?»* — так материал усвоится лучше.


## Установка и импорт библиотек
Если вы в Google Colab — раскомментируйте `!pip install`, локально — устанавливайте через `pip` или `conda`.

In [6]:
# !pip install pandas matplotlib statsmodels numpy

import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import statsmodels.api as sm

plt.rcParams["figure.figsize"] = (8,4)
print("Библиотеки импортированы. Можно работать!")

Библиотеки импортированы. Можно работать!


# 1. Зачем нужны прогнозы?
Прогнозирование — это способ заранее оценить **будущие** значения показателей:
- **Логистика**: сколько товара заказывать в следующем месяце?
- **Маркетинг**: каков будет спрос на акцию?
- **Экономика**: предсказать ВВП, инфляцию.
- **IT**: сколько серверов нужно через 3 месяца, если трафик растёт?

Если у нас **исторический временной ряд**, часто применяют **количественные** методы (MA, экспоненциальное сглаживание, регрессию). Если данных мало или будущее может сильно отличаться от прошлого — нужны **качественные** методы (эксперты, сценарии), и обычно сочетают оба подхода.


# 2. Количественные методы
Рассмотрим:
- **Скользящее среднее (MA)**
- **Экспоненциальное сглаживание** (SES, метод Хольта)
- **Трендовые регрессии** (линейная)

Для более сложных рядов (сезонность, праздники) есть ARIMA, Prophet и т.п. Но начнём с базовой логики.

## 2.1 Скользящее среднее
**Идея**: берём окно из \(N\) последних точек ряда, считаем их среднее — получаем сглаженное значение. 
Для прогноза на 1 шаг вперёд часто берут последнее сглаженное.

**Формула** простого MA: для момента \(t\) (если окно = \(N\)):
$$\displaystyle \text{MA}_t = \frac{1}{N} \sum_{i=0}^{N-1} Y_{t - i},$$
где \(Y_t\) — значение ряда в момент \(t\). В pandas это делается через `.rolling(N).mean()`.

**Отставание**: если в данных тренд, MA отстаёт. Чем шире окно, тем сильнее сглаживание, но и запаздывание. 

### Пример
Допустим, есть временной ряд продаж:
```
[120, 135, 128, 150, 160, 155, 170]
```
Возьмём окно=3.

In [ ]:
sales_data = pd.Series([120, 135, 128, 150, 160, 155, 170])
ma3 = sales_data.rolling(window=3).mean()
print("Скользящее среднее (окно=3):")
print(ma3)

ma_forecast = ma3.iloc[-1]  # Прогноз = последнее среднее
print("Прогноз на следующий шаг:", ma_forecast)

Попробуйте увеличить `window=5` и посмотрите, как значения станут более сглаженными.


## 2.2 Экспоненциальное сглаживание
Вместо «жёсткого» окна учитываем **всю** историю, но с убывающими весами для старых данных.

### Простейшее (SES)
Рекурсивная формула:
$$\displaystyle S_t = \alpha Y_t + (1 - \alpha) S_{t-1},$$
где \(0 < \alpha < 1\). Здесь \(Y_t\) — фактическое значение ряда в момент \(t\), а \(S_t\) — сглаженная оценка уровня. Параметр \(\alpha\) определяет, насколько быстро «забываются» старые данные:
- \(\alpha\approx 1\) → модель почти «копирует» последнее значение (чувствительна к шуму).
- \(\alpha\approx 0\) → сильное сглаживание, модель «помнит» долго, но может отставать.

### Пример
Возьмём ряд (дневной трафик):
```
[105,132,120,135,142,160,155,172,180,178]
```
и применим SES (\(\alpha = 0.5\)).

In [ ]:
traffic = pd.Series([105,132,120,135,142,160,155,172,180,178])
model_ses = SimpleExpSmoothing(traffic)
fit_ses = model_ses.fit(smoothing_level=0.5, optimized=False)

plt.plot(traffic, label="Исходный ряд", marker='o')
plt.plot(fit_ses.fittedvalues, label="SES(α=0.5)", marker='*')
plt.title("Пример экспоненциального сглаживания")
plt.legend()
plt.show()

forecast_next = fit_ses.forecast(1)
print("Прогноз на следующий шаг:", forecast_next.iloc[0])

Вы можете менять `smoothing_level=0.2, 0.8,...` и смотреть, как меняется траектория.

#### Проверка ошибки
Посмотрим MSE (Mean Squared Error) на тех данных, где у нас есть факты. Чем меньше, тем лучше.


In [ ]:
residuals = traffic - fit_ses.fittedvalues
mse = (residuals**2).mean()
print("MSE =", mse)

### Метод Хольта (двойное сглаживание)
Если ряд имеет **линейный тренд**, SES даёт горизонтальную экстраполяцию (\(S_t\)). Метод Хольта добавляет оценку тренда:
$$\displaystyle L_t = \alpha \, Y_t + (1-\alpha)(L_{t-1}+T_{t-1}),$$
$$\displaystyle T_t = \beta\,(L_t - L_{t-1}) + (1-\beta)\,T_{t-1},$$
где \( L_t \) — сглаженный уровень, \( T_t \) — сглаженный наклон (тренд), \( \alpha, \beta \in (0,1) \). Прогноз на \(h\) шагов: \(L_t + h\,T_t\).

Пример:

In [ ]:
fit_holt = Holt(traffic).fit(smoothing_level=0.3, smoothing_trend=0.1, optimized=False)
holt_forecast = fit_holt.forecast(3)
print("Метод Хольта (двойное сглаживание): прогноз на 3 шага:", list(holt_forecast))

Этот метод даёт **линейно** возрастающий (или убывающий) прогноз.
Если у вас ещё сезонность, есть Holt-Winters (тройное сглаживание).

## 2.3 Трендовые регрессии
Если предполагаем, что \(Y(t)\) описывается прямой, полиномом или экспонентой, строим **регрессию по времени**. 

### Пример: Линейная
Пусть есть 12 наблюдений (рост пользователей в млн):

In [ ]:
t_lin = np.arange(1,13)
users = np.array([1.2,1.5,2.1,3.0,4.2,5.5,7.1,8.9,10.5,11.8,12.6,13.0])

X_lin = sm.add_constant(t_lin)
model_lin = sm.OLS(users, X_lin).fit()
print(model_lin.summary())

# Прогноз на 6 месяцев вперёд:
t_future_lin = np.arange(13,19)
Xf_lin = sm.add_constant(t_future_lin)
pred_lin = model_lin.predict(Xf_lin)
print("Линейный прогноз для t=13..18:", pred_lin)

Если данные замедляются (S-образный рост), линейная может переоценить дальнее будущее. Можно попробовать **полиномиальную** (\(Y=a+bt+ct^2\)) или логистическую. 


# 3. Качественные методы
Когда мало исторических данных или будущее может быть другим, помогают **экспертные** и **сценарные** подходы:
- **Экспертные оценки**: опрос специалистов, их мнения можно усреднить или искать консенсус.
- **Метод Делфи**: многораундовый анонимный опрос, эксперты видят обобщённые результаты и корректируют ответы, формируя более надёжный консенсус.
- **Сценарный анализ**: рассматриваем несколько сценариев ("базовый", "оптимистичный", "пессимистичный"), учитывая неопределённости.
- **SWOT-анализ**: смотрим на Strengths/Weaknesses (внутренние) и Opportunities/Threats (внешние) компании.

Часто используют **комбинацию**: построили количественный прогноз → эксперты внесли коррективы → сформировали несколько сценариев.


# 4. Задачи (три уровня, по три задания)

Ниже набор заданий. Для каждого уровня (Простой, Стандартный, Продвинутый) даны по три подзадачи. Сначала попробуйте решить, затем смотрите **Решения**.

## Уровень «Простой»
1. **(A1)**: Дан ряд `[100,120,140,130,125]`. Найдите 2-периодное скользящее среднее (MA2) для каждой точки, начиная с 2-й, и постройте список этих средних.
2. **(A2)**: Дан `[120,135,128,150,160,155,170]`. Вычислите 3-периодное среднее для месяцев 3–7 и возьмите последнее как прогноз на месяц 8.
3. **(A3)**: Сравните 2-периодное и 3-периодное скользящее среднее (MA2 и MA3) на ряде `[110,130,125,140,145,150]`. Какой из них сильнее сглаживает и почему?

## Уровень «Стандартный»
1. **(B1)**: Дан ряд `[80,90,85,100,120,130,125,140]`. Найдите скользящее среднее с окном=3 и окном=4. Постройте график, сравните, где сглаживание сильнее, а где больше «запаздывание».
2. **(B2)**: Дан `[105,132,120,135,142,160,155,172,180,178]`. Сравните прогноз методом:
   - MA(3) (последние 3 точки);
   - SES(\(\alpha=0.5\)).
   Какой выше? Когда выбирают большее \(\alpha\)?
3. **(B3)**: Попробуйте на том же ряде `[105,132,120,135,142,160,155,172,180,178]` несколько значений \(\alpha\) (0.2, 0.5, 0.8) для SES. Подсчитайте MSE, где она минимальна?

## Уровень «Продвинутый»
1. **(C1)**: Постройте график MSE в зависимости от \(\alpha\) (от 0.1 до 0.9 с шагом 0.1) для ряда `[105,132,120,135,142,160,155,172,180,178]` (SES). Какое \(\alpha\) даёт наименьшую ошибку?
2. **(C2)**: Дан `[1.2,1.5,2.1,3.0,4.2,5.5,7.1,8.9,10.5,11.8,12.6,13.0]` (t=1..12). Постройте **линейную** регрессию, сделайте прогноз на t=13..18. Затем добавьте столбец \(t^2\) (квадратичная), сравните. Какая модель кажется ближе к реальности, если рост замедляется?
3. **(C3)**: Модифицируйте данные `[1.2,1.5,2.1,3.0,4.2,5.5,7.1,8.9,10.5,11.8,12.6,13.0]` так, чтобы последние 2 точки давали резкий **сплеск** (скажем, вместо 12.6,13.0 возьмите 16.0,18.0). Сравните, как сильно это меняет линейную регрессию и квадратичную. Кто реагирует сильнее?


# Решения
Ниже — решения всех подзадач. Сверяйтесь, если уже попробовали выполнить.

## Уровень «Простой»
### Задача A1
```
Дан ряд [100,120,140,130,125]. Найдите 2-периодное скользящее среднее.
```
**Решение**: MA2 означает усредняем каждые 2 подряд идущие точки.
- Для 2-й точки: (100+120)/2=110
- 3-й: (120+140)/2=130
- 4-й: (140+130)/2=135
- 5-й: (130+125)/2=127.5
Итого список MA2: `[110,130,135,127.5]` (для точек с 2-го по 5-й).

In [ ]:
valsA1 = pd.Series([100,120,140,130,125])
ma2_A1 = valsA1.rolling(2).mean()
print(ma2_A1)
print("Список MA2 (без NaN):", list(ma2_A1.dropna()))

### Задача A2
```
Дан [120,135,128,150,160,155,170]. Вычислить 3-периодное среднее (месяцы 3..7) и взять последнее как прогноз на 8.
```
**Решение**: 
- Месяц 3: (120+135+128)/3=127.67
- Месяц 4: 137.67,
- Месяц 5: 146,
- Месяц 6: 155,
- Месяц 7: 161.67.
Прогноз на 8: 161.67.


In [ ]:
valsA2 = pd.Series([120,135,128,150,160,155,170])
ma3_A2 = valsA2.rolling(3).mean()
print(ma3_A2)
print("Прогноз на 8 =", ma3_A2.iloc[-1])

### Задача A3
```
Сравните 2-периодное и 3-периодное среднее (MA2 vs MA3) на ряде [110,130,125,140,145,150].
Кто сильнее сглаживает?
```
**Решение**:
- MA2 опирается лишь на 2 последние точки, более «чувствительно» к колебаниям.
- MA3 учитывает 3 точки, сглаживает чуть сильнее.
Поэтому MA3 в целом будет плавнее, но может чуть больше отставать.


In [ ]:
valsA3 = pd.Series([110,130,125,140,145,150])
ma2_A3 = valsA3.rolling(2).mean()
ma3_A3 = valsA3.rolling(3).mean()
print("MA2:")
print(ma2_A3)
print("MA3:")
print(ma3_A3)

## Уровень «Стандартный»
### Задача B1
```
Ряд [80,90,85,100,120,130,125,140]. Найти MA(3) и MA(4), построить график.
Где сглаживание сильнее? Где больше запаздывание?
```
**Решение**:
- MA(4) = среднее из 4 последних точек, значит сгладит сильнее, но отставание больше.
- MA(3) сглаживает чуть меньше, но реагирует быстрее.


In [ ]:
valsB1 = pd.Series([80,90,85,100,120,130,125,140])
ma3_B1 = valsB1.rolling(3).mean()
ma4_B1 = valsB1.rolling(4).mean()
plt.plot(valsB1, label="Исходный", marker='o')
plt.plot(ma3_B1, label="MA(3)", marker='*')
plt.plot(ma4_B1, label="MA(4)", marker='x')
plt.legend()
plt.title("Задача B1: сравнение MA(3) и MA(4)")
plt.show()

### Задача B2
```
Ряд [105,132,120,135,142,160,155,172,180,178]. Сравните:
 - MA(3) (последние 3 точки)
 - SES(α=0.5)
Какой выше?
```
**Решение**:
- MA(3) обычно ближе к последним точкам (например, если последние были 172,180,178, среднее=176.67).
- SES(0.5) учитывает также старые точки, сглаживает ~ 174.4.
MA(3) выше.


In [ ]:
valsB2 = pd.Series([105,132,120,135,142,160,155,172,180,178])

ma3_B2 = valsB2.rolling(3).mean().iloc[-1]
print("MA(3) =", ma3_B2)

modelB2 = SimpleExpSmoothing(valsB2)
fitB2 = modelB2.fit(smoothing_level=0.5, optimized=False)
sesB2 = fitB2.forecast(1).iloc[0]
print("SES(0.5) =", sesB2)

### Задача B3
```
На том же ряде [105,132,120,135,142,160,155,172,180,178] возьмите SES с α=0.2, 0.5, 0.8,
и посчитайте MSE. Где минимальна?
```
**Решение**: запускаем несколько раз. 
- Если ряд растёт, большой \(\alpha\) может лучше «догонять» тренд, но и ловить шум.
- Нужно проверить, где MSE меньше.


In [ ]:
valsB3 = pd.Series([105,132,120,135,142,160,155,172,180,178])
alphas = [0.2, 0.5, 0.8]
for a in alphas:
    fit = SimpleExpSmoothing(valsB3).fit(smoothing_level=a, optimized=False)
    mse_ = ((valsB3 - fit.fittedvalues)**2).mean()
    print(f"alpha={a}, MSE={mse_:.2f}")

# Уровень «Продвинутый»
### Задача C1
```
Постройте график MSE в зависимости от alpha (0.1..0.9) на ряде [105,132,120,135,142,160,155,172,180,178] (SES).
Какое alpha минимизирует ошибку?
```
**Решение**: перебираем \(\alpha\) от 0.1 до 0.9, шаг 0.1, считаем MSE, строим график.


In [ ]:
valsC1 = pd.Series([105,132,120,135,142,160,155,172,180,178])
alphas_c1 = np.arange(0.1,1.0,0.1)
mses_c1 = []
for a in alphas_c1:
    fit = SimpleExpSmoothing(valsC1).fit(smoothing_level=a, optimized=False)
    mse_ = ((valsC1 - fit.fittedvalues)**2).mean()
    mses_c1.append(mse_)

plt.plot(alphas_c1, mses_c1, marker='o')
plt.xlabel("alpha")
plt.ylabel("MSE")
plt.title("Задача C1: MSE vs alpha")
plt.show()

min_mse = min(mses_c1)
best_alpha = alphas_c1[mses_c1.index(min_mse)]
print("Лучшее alpha=", best_alpha, ", MSE=", min_mse)

Возможно, получится что-то вроде \(\alpha=0.5\) или 0.6. Каждый ряд индивидуален.


### Задача C2
```
Ряд [1.2,1.5,2.1,3.0,4.2,5.5,7.1,8.9,10.5,11.8,12.6,13.0] (t=1..12).
1) Линейная регрессия -> прогноз t=13..18.
2) Добавить t^2 (квадрат), сравнить.
3) Если к концу замедление, какая модель реальнее?
```
**Решение**:
См. код ниже. Линейная растёт равномерно, квадратичная может ускоряться или замедляться в зависимости от знака \(c\). Если рост замедляется, а \(c>0\), квадратичная может переоценить. Наверняка логистическая была бы лучше, но здесь только сравниваем 2 варианта.


In [ ]:
tC2 = np.arange(1,13)
yC2 = np.array([1.2,1.5,2.1,3.0,4.2,5.5,7.1,8.9,10.5,11.8,12.6,13.0])

# Линейная
X_linC2 = sm.add_constant(tC2)
model_linC2 = sm.OLS(yC2, X_linC2).fit()
t_futureC2 = np.arange(13,19)
Xf_linC2 = sm.add_constant(t_futureC2)
pred_linC2 = model_linC2.predict(Xf_linC2)
print("Линейная (a,b):", model_linC2.params)
print("Лин. прогноз:", pred_linC2)

# Квадратичная
tC2sq = tC2**2
X_quadC2 = np.column_stack((tC2, tC2sq))
X_quadC2 = sm.add_constant(X_quadC2)
model_quadC2 = sm.OLS(yC2, X_quadC2).fit()
t_futureC2sq = t_futureC2**2
Xf_quadC2 = np.column_stack((t_futureC2, t_futureC2sq))
Xf_quadC2 = sm.add_constant(Xf_quadC2)
pred_quadC2 = model_quadC2.predict(Xf_quadC2)
print("Квадратичная (a,b,c):", model_quadC2.params)
print("Квадрат. прогноз:", pred_quadC2)

### Задача C3
```
В том же ряде [1.2,1.5,2.1,...,12.6,13.0], замените последние 2 точки (12.6,13.0)
на (16.0,18.0) - резкий всплеск.
Сравните, как сильно меняется лин. и квадр. регрессия.
Кто реагирует сильнее?
```
**Решение**:
Если последние точки стали намного больше, линейная поднимется, но квадратичная ещё сильнее может «подтянуться», особенно если \(c>0\). Т.к. в квадратике последние t имеют большýю «силу». Значит, квадратичная модель может реагировать сильнее.


In [ ]:
tC3 = np.arange(1,13)
yC3 = np.array([1.2,1.5,2.1,3.0,4.2,5.5,7.1,8.9,10.5,11.8,16.0,18.0])  # заменили

# Линейная
X_linC3 = sm.add_constant(tC3)
model_linC3 = sm.OLS(yC3, X_linC3).fit()
print("Линейная params:", model_linC3.params)

# Квадратичная
tC3sq = tC3**2
X_quadC3 = np.column_stack((tC3, tC3sq))
X_quadC3 = sm.add_constant(X_quadC3)
model_quadC3 = sm.OLS(yC3, X_quadC3).fit()
print("Квадратичная params:", model_quadC3.params)

Можно сравнить со старым результатом (без всплеска). Квадратичная кривая может существенно «переоценить».


# Дополнительные ресурсы
- Изучите **Holt-Winters** (тройное экспоненциальное сглаживание) для сезонных рядов.
- Попробуйте **pmdarima** (Auto-ARIMA) или **fbprophet** (Prophet) для более сложных рядов.
- Для качественных методов (Delphi, сценарии) есть много литературы по «форсайту».